# Implementation
<div id="wave:2D3D:impl"></div>

[Video here](mov-wave/mayavi/wave2D_u0_gaussian/movie.mp4)

We shall now describe in detail various Python implementations
for solving a standard 2D, linear wave equation with constant
wave velocity and $u=0$ on the
boundary. The wave equation is to be solved
in the space-time domain $\Omega\times (0,T]$,
where $\Omega = (0,L_x)\times (0,L_y)$ is a rectangular spatial
domain. More precisely,
the complete initial-boundary value problem is defined by

<!-- Equation labels as ordinary links -->
<div id="_auto1"></div>

$$
\begin{equation}
u_{tt} = c^2(u_{xx} + u_{yy}) + f(x,y,t),\quad (x,y)\in \Omega,\ t\in (0,T],
\label{_auto1} \tag{1}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto2"></div>

$$
\begin{equation} 
u(x,y,0) = I(x,y),\quad (x,y)\in\Omega,
\label{_auto2} \tag{2}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto3"></div>

$$
\begin{equation} 
u_t(x,y,0) = V(x,y),\quad (x,y)\in\Omega,
\label{_auto3} \tag{3}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto4"></div>

$$
\begin{equation} 
u = 0,\quad (x,y)\in\partial\Omega,\ t\in (0,T],
\label{_auto4} \tag{4}
\end{equation}
$$

where $\partial\Omega$ is the boundary of $\Omega$, in this case
the four sides of the rectangle $\Omega = [0,L_x]\times [0,L_y]$:
$x=0$, $x=L_x$, $y=0$, and $y=L_y$.

The PDE is discretized as

$$
[D_t D_t u = c^2(D_xD_x u + D_yD_y u) + f]^n_{i,j},
$$

which leads to an explicit updating formula to be implemented in a
program:

$$
u^{n+1}_{i,j} = -u^{n-1}_{i,j} + 2u^n_{i,j} + \nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="wave:2D3D:impl1:2Du0:ueq:discrete"></div>

$$
\begin{equation} 
\quad C_x^2(
u^{n}_{i+1,j} - 2u^{n}_{i,j} + u^{n}_{i-1,j}) + C_y^2
(u^{n}_{i,j+1} - 2u^{n}_{i,j} + u^{n}_{i,j-1}) + \Delta t^2 f_{i,j}^n,
\label{wave:2D3D:impl1:2Du0:ueq:discrete} \tag{5}
\end{equation}
$$

for all interior mesh points $i\in\seti{\mathcal{I}_x}$ and
$j\in\seti{\Iy}$, for $n\in\setr{\mathcal{I}_t}$.
The constants $C_x$ and $C_y$ are defined as

$$
C_x = c\frac{\Delta t}{\Delta x},\quad C_y = c\frac{\Delta t}{\Delta y}
\thinspace .
$$

At the boundary, we simply set $u^{n+1}_{i,j}=0$ for
$i=0$, $j=0,\ldots,N_y$; $i=N_x$, $j=0,\ldots,N_y$;
$j=0$, $i=0,\ldots,N_x$; and $j=N_y$, $i=0,\ldots,N_x$.
For the first step, $n=0$, ([5](#wave:2D3D:impl1:2Du0:ueq:discrete))
is combined with the discretization of the initial condition $u_t=V$,
$[D_{2t} u = V]^0_{i,j}$ to obtain a special formula for
$u^1_{i,j}$ at the interior mesh points:

$$
u^{1}_{i,j} = u^0_{i,j} + \Delta t V_{i,j} + \nonumber
$$

$$
\quad {\frac{1}{2}}C_x^2(
u^{0}_{i+1,j} - 2u^{0}_{i,j} + u^{0}_{i-1,j}) + {\frac{1}{2}}C_y^2
(u^{0}_{i,j+1} - 2u^{0}_{i,j} + u^{0}_{i,j-1}) +\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="wave:2D3D:impl1:2Du0:ueq:n0:discrete"></div>

$$
\begin{equation} 
\quad \frac{1}{2}\Delta t^2f_{i,j}^n,
\label{wave:2D3D:impl1:2Du0:ueq:n0:discrete} \tag{6}
\end{equation}
$$

The algorithm is very similar to the one in 1D:

1. Set initial condition $u^0_{i,j}=I(x_i,y_j)$

2. Compute $u^1_{i,j}$ from ([5](#wave:2D3D:impl1:2Du0:ueq:discrete))

3. Set $u^1_{i,j}=0$ for the boundaries $i=0,N_x$, $j=0,N_y$

4. For $n=1,2,\ldots,N_t$:

a. Find $u^{n+1}_{i,j}$ from ([5](#wave:2D3D:impl1:2Du0:ueq:discrete))
    for all internal mesh points, $i\in\seti{\mathcal{I}_x}$, $j\in\seti{\Iy}$

b. Set $u^{n+1}_{i,j}=0$ for the boundaries $i=0,N_x$, $j=0,N_y$


## Scalar computations
<div id="wave2D3D:impl:scalar"></div>


The `solver` function for a 2D case with constant wave velocity and
boundary condition $u=0$ is analogous to the 1D case with similar parameter
values (see `wave1D_u0.py`), apart from a few necessary
extensions. The code is found in the program
[`wave2D_u0.py`](${src_wave}/wave2D_u0/wave2D_u0.py).

### Domain and mesh

The spatial domain is now $[0,L_x]\times [0,L_y]$, specified
by the arguments `Lx` and `Ly`. Similarly, the number of mesh
points in the $x$ and $y$ directions,
$N_x$ and $N_y$, become the arguments `Nx` and `Ny`.
In multi-dimensional problems it makes less sense to specify a
Courant number since the wave velocity is a vector and mesh spacings
may differ in the various spatial directions.
We therefore give $\Delta t$ explicitly. The signature of
the `solver` function is then

In [3]:
def solver(I, V, f, c, Lx, Ly, Nx, Ny, dt, T,
           user_action=None, version='scalar'):

SyntaxError: unexpected EOF while parsing (<ipython-input-3-edc2d0ac531f>, line 2)

Key parameters used in the calculations are created as

In [4]:
x = linspace(0, Lx, Nx+1)                  # mesh points in x dir
y = linspace(0, Ly, Ny+1)                  # mesh points in y dir
dx = x[1] - x[0]
dy = y[1] - y[0]
Nt = int(round(T/float(dt)))
t = linspace(0, N*dt, N+1)                 # mesh points in time
Cx2 = (c*dt/dx)**2;  Cy2 = (c*dt/dy)**2    # help variables
dt2 = dt**2

NameError: name 'linspace' is not defined

### Solution arrays

We store $u^{n+1}_{i,j}$, $u^{n}_{i,j}$, and
$u^{n-1}_{i,j}$ in three two-dimensional arrays,

In [5]:
u   = zeros((Nx+1,Ny+1))   # solution array
u_n = [zeros((Nx+1,Ny+1)), zeros((Nx+1,Ny+1))]  # t-dt, t-2*dt

NameError: name 'zeros' is not defined

where $u^{n+1}_{i,j}$ corresponds to `u[i,j]`,
$u^{n}_{i,j}$ to `u_n[i,j]`, and
$u^{n-1}_{i,j}$ to `u_nm1[i,j]`.


### Index sets

mathcal{I}_t is also convenient to introduce the index sets (cf. the section [wave:indexset](#wave:indexset))

In [6]:
mathcal{I}_x = range(0, u.shape[0])
Iy = range(0, u.shape[1])
mathcal{I}_t = range(0, t.shape[0])

SyntaxError: invalid syntax (<ipython-input-6-a6a302513643>, line 1)

### Computing the solution

Inserting the initial
condition `I` in `u_n` and making a callback to the user in terms of
the `user_action` function is a straightforward generalization of
the 1D code from the section [wave:string:impl](#wave:string:impl):

In [7]:
for i in mathcal{I}_x:
    for j in Iy:
        u_n[i,j] = I(x[i], y[j])

if user_action is not None:
    user_action(u_n, x, xv, y, yv, t, 0)

SyntaxError: invalid syntax (<ipython-input-7-f7875c171ca8>, line 1)

The `user_action` function has additional arguments compared to the
1D case. The arguments `xv` and `yv` will be commented
upon in the section [Vectorized computations](#wave2D3D:impl:vectorized).

The key finite difference formula ([wave:2D3D:models:unp1](#wave:2D3D:models:unp1))
for updating the solution at
a time level is implemented in a separate function as

In [8]:
def advance_scalar(u, u_n, u_nm1, f, x, y, t, n, Cx2, Cy2, dt2,
                   V=None, step1=False):
    mathcal{I}_x = range(0, u.shape[0]);  Iy = range(0, u.shape[1])
    if step1:
        dt = sqrt(dt2)  # save
        Cx2 = 0.5*Cx2;  Cy2 = 0.5*Cy2; dt2 = 0.5*dt2  # redefine
        D1 = 1;  D2 = 0
    else:
        D1 = 2;  D2 = 1
    for i in mathcal{I}_x[1:-1]:
        for j in Iy[1:-1]:
            u_xx = u_n[i-1,j] - 2*u_n[i,j] + u_n[i+1,j]
            u_yy = u_n[i,j-1] - 2*u_n[i,j] + u_n[i,j+1]
            u[i,j] = D1*u_n[i,j] - D2*u_nm1[i,j] + \
                     Cx2*u_xx + Cy2*u_yy + dt2*f(x[i], y[j], t[n])
            if step1:
                u[i,j] += dt*V(x[i], y[j])
    # Boundary condition u=0
    j = Iy[0]
    for i in mathcal{I}_x: u[i,j] = 0
    j = Iy[-1]
    for i in mathcal{I}_x: u[i,j] = 0
    i = mathcal{I}_x[0]
    for j in Iy: u[i,j] = 0
    i = mathcal{I}_x[-1]
    for j in Iy: u[i,j] = 0
    return u

SyntaxError: invalid syntax (<ipython-input-8-41d61586e2d2>, line 3)

The `step1` variable has been introduced to allow the formula to be
reused for the first step, computing $u^1_{i,j}$:

In [9]:
u = advance_scalar(u, u_n, f, x, y, t,
                   n, Cx2, Cy2, dt, V, step1=True)

NameError: name 'advance_scalar' is not defined

Below, we will make many alternative implementations of the
`advance_scalar` function to speed up the code since most of
the CPU time in simulations is spent in this function.

**Remark: How to use the solution.**

The `solver` function in the `wave2D_u0.py` code
updates arrays for the next time step by switching references as
described in the section [wave:pde1:impl:ref:switch](#wave:pde1:impl:ref:switch). Any use of `u` on the
user's side is assumed to take place in the user action function. However,
should the code be changed such that `u` is returned and used as solution,
have in mind that you must return `u_n` after the time limit, otherwise
a `return u` will actually return `u_nm1` (due to the switching of array
indices in the loop)!



## Vectorized computations
<div id="wave2D3D:impl:vectorized"></div>


The scalar code above turns out to be extremely slow for large 2D
meshes, and probably useless in 3D beyond debugging of small test cases.
Vectorization is therefore a must for multi-dimensional
finite difference computations in Python. For example,
with a mesh consisting of $30\times 30$ cells, vectorization
brings down the CPU time by a factor of 70 (!). Equally important,
vectorized code can also easily be parallelized to take (usually)
optimal advantage of parallel computer platforms.

In the vectorized case, we must be able to evaluate user-given
functions like $I(x,y)$ and $f(x,y,t)$ for the entire mesh in one
operation (without loops). These user-given functions are provided as
Python functions `I(x,y)` and `f(x,y,t)`, respectively.  Having the
one-dimensional coordinate arrays `x` and `y` is not sufficient when
calling `I` and `f` in a vectorized way.  We must extend `x` and `y`
to their vectorized versions `xv` and `yv`:

In [10]:
from numpy import newaxis
xv = x[:,newaxis]
yv = y[newaxis,:]
# or
xv = x.reshape((x.size, 1))
yv = y.reshape((1, y.size))

NameError: name 'x' is not defined

This is a standard required technique when evaluating functions over
a 2D mesh, say `sin(xv)*cos(xv)`, which then gives a result with shape
`(Nx+1,Ny+1)`. Calling `I(xv, yv)` and `f(xv, yv, t[n])` will now
return `I` and `f` values for the entire set of mesh points.

With the `xv` and `yv` arrays for vectorized computing,
setting the initial condition is just a matter of

In [11]:
u_n[:,:] = I(xv, yv)

NameError: name 'I' is not defined

One could also have written `u_n = I(xv, yv)` and let `u_n` point to a
new object, but vectorized operations often make use of direct
insertion in the original array through `u_n[:,:]`, because sometimes
not all of the array is to be filled by such a function
evaluation. This is the case with the computational scheme for
$u^{n+1}_{i,j}$:

In [12]:
def advance_vectorized(u, u_n, u_nm1, f_a, Cx2, Cy2, dt2,
                       V=None, step1=False):
    if step1:
        dt = np.sqrt(dt2)  # save
        Cx2 = 0.5*Cx2;  Cy2 = 0.5*Cy2; dt2 = 0.5*dt2  # redefine
        D1 = 1;  D2 = 0
    else:
        D1 = 2;  D2 = 1
    u_xx = u_n[:-2,1:-1] - 2*u_n[1:-1,1:-1] + u_n[2:,1:-1]
    u_yy = u_n[1:-1,:-2] - 2*u_n[1:-1,1:-1] + u_n[1:-1,2:]
    u[1:-1,1:-1] = D1*u_n[1:-1,1:-1] - D2*u_nm1[1:-1,1:-1] + \
                   Cx2*u_xx + Cy2*u_yy + dt2*f_a[1:-1,1:-1]
    if step1:
        u[1:-1,1:-1] += dt*V[1:-1, 1:-1]
    # Boundary condition u=0
    j = 0
    u[:,j] = 0
    j = u.shape[1]-1
    u[:,j] = 0
    i = 0
    u[i,:] = 0
    i = u.shape[0]-1
    u[i,:] = 0
    return u

Array slices in 2D are more complicated to understand than those in
1D, but the logic from 1D applies to each dimension separately.
For example, when doing $u^{n}_{i,j} - u^{n}_{i-1,j}$ for $i\in\setr{\mathcal{I}_x}$,
we just keep `j` constant and make a slice in the first index:
`u_n[1:,j] - u_n[:-1,j]`, exactly as in 1D. The `1:` slice
specifies all the indices $i=1,2,\ldots,N_x$ (up to the last
valid index),
while `:-1` specifies the relevant indices for the second term:
$0,1,\ldots,N_x-1$ (up to, but not including the last index).

In the above code segment, the situation is slightly more complicated,
because each displaced slice in one direction is
accompanied by a `1:-1` slice in the other direction. The reason is
that we only work with the internal points for the index that is
kept constant in a difference.

The boundary conditions along the four sides make use of
a slice consisting of all indices along a boundary:

In [13]:
u[: ,0] = 0
u[:,Ny] = 0
u[0 ,:] = 0
u[Nx,:] = 0

NameError: name 'u' is not defined

In the vectorized update of `u` (above), the function `f` is first computed
as an array over all mesh points:

        f_a = f(xv, yv, t[n])


We could, alternatively, have used the call `f(xv, yv, t[n])[1:-1,1:-1]`
in the last term of the update statement, but other implementations
in compiled languages benefit from having `f` available in an array
rather than calling our Python function `f(x,y,t)` for
every point.

Also in the `advance_vectorized` function we have introduced a
boolean `step1` to reuse the formula for the first time step
in the same way as we did with `advance_scalar`.
We refer to the `solver` function in `wave2D_u0.py`
for the details on how the overall algorithm is implemented.

The callback function now has the arguments
`u, x, xv, y, yv, t, n`. The inclusion of `xv` and `yv` makes it
easy to, e.g., compute an exact 2D solution in the callback function
and compute errors, through an expression like
`u - u_exact(xv, yv, t[n])`.

## Verification
<div id="wave2D3D:impl:verify"></div>

### Testing a quadratic solution

The 1D solution from the section [wave:pde2:fd:verify:quadratic](#wave:pde2:fd:verify:quadratic) can be
generalized to multi-dimensions and provides a test case where the
exact solution also fulfills the discrete equations, such that we know
(to machine precision) what numbers the solver function should
produce. In 2D we use the following generalization of
([wave:pde2:fd:verify:quadratic:uex](#wave:pde2:fd:verify:quadratic:uex)):

<!-- Equation labels as ordinary links -->
<div id="wave2D3D:impl:verify:quadratic"></div>

$$
\begin{equation}
\uex(x,y,t) = x(L_x-x)y(L_y-y)(1+{\frac{1}{2}}t)
\thinspace .
\label{wave2D3D:impl:verify:quadratic} \tag{7}
\end{equation}
$$

This solution fulfills the PDE problem if $I(x,y)=\uex(x,y,0)$,
$V=\frac{1}{2}\uex(x,y,0)$, and $f=2c^2(1+{\frac{1}{2}}t)(y(L_y-y) +
x(L_x-x))$. To show that $\uex$ also solves the discrete equations,
we start with the general results $[D_t D_t 1]^n=0$, $[D_t D_t t]^n=0$,
and $[D_t D_t t^2]=2$, and use these to compute

$$
\begin{align*}
[D_xD_x \uex]^n_{i,j} &= [y(L_y-y)(1+{\frac{1}{2}}t) D_xD_x x(L_x-x)]^n_{i,j}\\
&= y_j(L_y-y_j)(1+{\frac{1}{2}}t_n)(-2)\thinspace .
\end{align*}
$$

A similar calculation must be carried out for the $[D_yD_y
\uex]^n_{i,j}$ and $[D_tD_t \uex]^n_{i,j}$ terms.  One must also show
that the quadratic solution fits the special formula for
$u^1_{i,j}$. The details are left as [Exercise 1: Check that a solution fulfills the discrete model](#wave:exer:quadratic:2D).
The `test_quadratic` function in the
[`wave2D_u0.py`](${src_wave}/wave2D_u0/wave2D_u0.py)
program implements this verification as a proper test function
for the pytest and nose frameworks.

## Visualization

Eventually, we are ready for a real application with our code!
Look at the `wave2D_u0.py` and the `gaussian` function. mathcal{I}_t
starts with a Gaussian function to see how it propagates in a square
with $u=0$ on the boundaries:

In [14]:
def gaussian(plot_method=2, version='vectorized', save_plot=True):
    """
    Initial Gaussian bell in the middle of the domain.
    plot_method=1 applies mesh function,
    =2 means surf, =3 means Matplotlib, =4 means mayavi,
    =0 means no plot.
    """
    # Clean up plot files
    for name in glob('tmp_*.png'):
        os.remove(name)

    Lx = 10
    Ly = 10
    c = 1.0

    from numpy import exp

    def I(x, y):
        """Gaussian peak at (Lx/2, Ly/2)."""
        return exp(-0.5*(x-Lx/2.0)**2 - 0.5*(y-Ly/2.0)**2)

    def plot_u(u, x, xv, y, yv, t, n):
        """User action function for plotting."""
	...

    Nx = 40; Ny = 40; T = 20
    dt, cpu = solver(I, None, None, c, Lx, Ly, Nx, Ny, -1, T,
                     user_action=plot_u, version=version)

TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-14-c5fdd8482238>, line 24)

### Matplotlib

We want to animate a 3D surface in Matplotlib, but this is a really
slow process and not recommended, so we consider Matplotlib not an
option as long as on-screen animation is desired. One can use the
recipes for single shots of $u$, where it does produce high-quality
3D plots.


### Gnuplot

Let us look at different ways for visualization. We import SciTools as
`st` and can access `st.mesh` and `st.surf` in Matplotlib or Gnuplot,
but this is not supported except for the Gnuplot package, where it
works really well ([Figure](#wave2D3D:impl:viz:fig:gnuplot1)).  Then
we choose `plot_method=2` (or less relevant `plot_method=1`) and force
the backend for SciTools to be Gnuplot (if you have the C package
Gnuplot and the `Gnuplot.py` Python interface module installed):

        Terminal> python wave2D_u0.py --SCITOOLS_easyviz_backend gnuplot


mathcal{I}_t gives a nice visualization with lifted surface and contours beneath.
[Figure](#wave2D3D:impl:viz:fig:gnuplot1) shows four plots of $u$.

<!-- dom:FIGURE: [fig-wave/wave2D_u0_gnuplot_gaussian.png, width=800 frac=1] Snapshots of the surface plotted by Gnuplot. <div id="wave2D3D:impl:viz:fig:gnuplot1"></div> -->
<!-- begin figure -->
<div id="wave2D3D:impl:viz:fig:gnuplot1"></div>

<p>Snapshots of the surface plotted by Gnuplot.</p>
<img src="fig-wave/wave2D_u0_gnuplot_gaussian.png" width=800>

<!-- end figure -->


Video files can be made of the PNG frames:

        Terminal> ffmpeg -i tmp_%04d.png -r 25 -vcodec flv movie.flv
        Terminal> ffmpeg -i tmp_%04d.png -r 25 -vcodec linx264 movie.mp4
        Terminal> ffmpeg -i tmp_%04d.png -r 25 -vcodec libvpx movie.webm
        Terminal> ffmpeg -i tmp_%04d.png -r 25 -vcodec libtheora movie.ogg


mathcal{I}_t is wise to use a high frame rate - a low one will just skip many
frames. There may also be considerable quality differences between the
different formats.

<!-- dom:MOVIE: [https://raw.githubusercontent.com/hplgit/fdm-book/master/doc/pub/book/html/mov-wave/gnuplot/wave2D_u0_gaussian/movie25.mp4] -->
<!-- begin movie -->

In [15]:
from IPython.display import HTML
_s = """
<div>
<video  loop controls width='640' height='365' preload='none'>
    <source src='https://raw.githubusercontent.com/hplgit/fdm-book/master/doc/pub/book/html/mov-wave/gnuplot/wave2D_u0_gaussian/movie25.mp4'  type='video/mp4;  codecs="avc1.42E01E, mp4a.40.2"'>
    <source src='https://raw.githubusercontent.com/hplgit/fdm-book/master/doc/pub/book/html/mov-wave/gnuplot/wave2D_u0_gaussian/movie25.webm' type='video/webm; codecs="vp8, vorbis"'>
    <source src='https://raw.githubusercontent.com/hplgit/fdm-book/master/doc/pub/book/html/mov-wave/gnuplot/wave2D_u0_gaussian/movie25.ogg'  type='video/ogg;  codecs="theora, vorbis"'>
</video>
</div>
<p><em></em></p>

<!-- Issue warning if in a Safari browser -->
<script language="javascript">
if (!!(window.safari)) {
  document.write("<div style=\"width: 95%%; padding: 10px; border: 1px solid #100; border-radius: 4px;\"><p><font color=\"red\">The above movie will not play in Safari - use Chrome, Firefox, or Opera.</font></p></div>")}
</script>

"""
HTML(_s)

<!-- end movie -->



### Mayavi

The best option for doing visualization of 2D and 3D scalar and vector fields
in Python programs is Mayavi, which is an interface to the high-quality
package VTK in C++. There is good online documentation and also
an introduction in Chapter 5 of [[Langtangen_2012]](#Langtangen_2012).

To obtain Mayavi on Ubuntu platforms you can write

        pip install mayavi --upgrade


For Mac OS X and Windows, we recommend using Anaconda.
To obtain Mayavi for Anaconda you can write

        conda install mayavi


Mayavi has a MATLAB-like interface called `mlab`. We can do

In [16]:
import mayavi.mlab as plt
# or
from mayavi import mlab

ModuleNotFoundError: No module named 'mayavi'

and have `plt` (as usual) or `mlab`
as a kind of MATLAB visualization access inside our program (just
more powerful and with higher visual quality).

The official documentation of the `mlab` module is provided in two
places, one for the [basic functionality](http://docs.enthought.com/mayavi/mayavi/auto/mlab_helper_functions.html)
and one for [further functionality](http://docs.enthought.com/mayavi/mayavi/auto/mlab_other_functions.html).
Basic [figure
handling](http://docs.enthought.com/mayavi/mayavi/auto/mlab_figure.html)
is very similar to the one we know from Matplotlib.  Just as for
Matplotlib, all plotting commands you do in `mlab` will go into the
same figure, until you manually change to a new figure.

Back to our application, the following code for the user action
function with plotting in Mayavi is relevant to add.

In [17]:
%matplotlib inline

# Top of the file
try:
    import mayavi.mlab as mlab
except:
    # We don't have mayavi
    pass

def solver(...):
    ...

def gaussian(...):
    ...
    if plot_method == 3:
        from mpl_toolkits.mplot3d import axes3d
        import matplotlib.pyplot as plt
        from matplotlib import cm
        plt.ion()
        fig = plt.figure()
        u_surf = None

    def plot_u(u, x, xv, y, yv, t, n):
        """User action function for plotting."""
        if t[n] == 0:
            time.sleep(2)
        if plot_method == 1:
            # Works well with Gnuplot backend, not with Matplotlib
            st.mesh(x, y, u, title='t=%g' % t[n], zlim=[-1,1],
                    caxis=[-1,1])
        elif plot_method == 2:
            # Works well with Gnuplot backend, not with Matplotlib
            st.surfc(xv, yv, u, title='t=%g' % t[n], zlim=[-1, 1],
                  colorbar=True, colormap=st.hot(), caxis=[-1,1],
                  shading='flat')
        elif plot_method == 3:
            print 'Experimental 3D matplotlib...not recommended'
        elif plot_method == 4:
	    # Mayavi visualization
            mlab.clf()
            extent1 = (0, 20, 0, 20,-2, 2)
            s = mlab.surf(x , y, u,
                          colormap='Blues',
                          warp_scale=5,extent=extent1)
            mlab.axes(s, color=(.7, .7, .7), extent=extent1,
                      ranges=(0, 10, 0, 10, -1, 1),
                      xlabel='', ylabel='', zlabel='',
                      x_axis_visibility=False,
                      z_axis_visibility=False)
            mlab.outline(s, color=(0.7, .7, .7), extent=extent1)
            mlab.text(6, -2.5, '', z=-4, width=0.14)
            mlab.colorbar(object=None, title=None,
                          orientation='horizontal',
                          nb_labels=None, nb_colors=None,
                          label_fmt=None)
            mlab.title('Gaussian t=%g' % t[n])
            mlab.view(142, -72, 50)
            f = mlab.gcf()
            camera = f.scene.camera
            camera.yaw(0)

        if plot_method > 0:
            time.sleep(0) # pause between frames
            if save_plot:
                filename = 'tmp_%04d.png' % n
		if plot_method == 4:
                    mlab.savefig(filename)  # time consuming!
		elif plot_method in (1,2):
                    st.savefig(filename)  # time consuming!

SyntaxError: invalid syntax (<ipython-input-17-35bddf66763c>, line 10)

This is a point to get started - visualization is as always a very
time-consuming and experimental discipline. With the PNG files we
can use `ffmpeg` to create videos.

<!-- dom:FIGURE: [fig-wave/mayavi2D_gaussian1.png, width=600 frac=0.8] Plot with Mayavi. -->
<!-- begin figure -->

<p>Plot with Mayavi.</p>
<img src="fig-wave/mayavi2D_gaussian1.png" width=600>

<!-- end figure -->


<!-- dom:MOVIE: [https://github.com/hplgit/fdm-book/blob/master/doc/pub/book/html/mov-wave/mayavi/wave2D_u0_gaussian/movie.mp4] -->
<!-- begin movie -->

In [18]:
# from IPython.display import HTML

# _s = """
# <div>
# <video  loop controls width='640' height='365' preload='none'>
#     <source src='mov-wave/mayavi/wave2D_u0_gaussian/movie.mp4'  type='video/mp4;  codecs="avc1.42E01E, mp4a.40.2"'>
# </video>
# </div>
# <p><em></em></p>

# <!-- Issue warning if in a Safari browser -->
# <script language="javascript">
# if (!!(window.safari)) {
#   document.write("<div style=\"width: 95%%; padding: 10px; border: 1px solid #100; border-radius: 4px;\"><p><font color=\"red\">The above movie will not play in Safari - use Chrome, Firefox, or Opera.</font></p></div>")}
# </script>

# """
# #     <source src='https://github.com/hplgit/fdm-book/blob/master/doc/pub/book/html/mov-wave/mayavi/wave2D_u0_gaussian/movie.ogg'  type='video/ogg;  codecs="theora, vorbis"'>
# HTML(_s)

[Video here](mov-wave/mayavi/wave2D_u0_gaussian/movie.mp4)

<!-- end movie -->



# Exercises



<!-- --- begin exercise --- -->

## Exercise 1: Check that a solution fulfills the discrete model
<div id="wave:exer:quadratic:2D"></div>

Carry out all mathematical details to show that
([7](#wave2D3D:impl:verify:quadratic)) is indeed a solution of the
discrete model for a 2D wave equation with $u=0$ on the boundary.
One must check the boundary conditions, the initial conditions,
the general discrete equation at a time level and the special
version of this equation for the first time level.
Filename: `check_quadratic_solution`.

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Project 2: Calculus with 2D mesh functions
<div id="wave:exer:mesh3D:calculus"></div>

The goal of this project is to redo
[wave:exer:mesh1D:calculus](#wave:exer:mesh1D:calculus) with 2D
mesh functions ($f_{i,j}$).

**Differentiation.**
The differentiation results in a discrete gradient
function, which in the 2D case can be represented by a three-dimensional
array `df[d,i,j]` where `d` represents the direction of
the derivative, and `i,j` is a mesh point in 2D.
Use centered differences for
the derivative at inner points and one-sided forward or backward
differences at the boundary points. Construct unit tests and
write a corresponding test function.

**Integration.**
The integral of a 2D mesh function $f_{i,j}$ is defined as

$$
F_{i,j} = \int_{y_0}^{y_j} \int_{x_0}^{x_i} f(x,y)dxdy,
$$

where $f(x,y)$ is a function that takes on the values of the
discrete mesh function $f_{i,j}$ at the mesh points, but can also
be evaluated in between the mesh points. The particular variation
between mesh points can be taken as bilinear, but this is not
important as we will use a product Trapezoidal rule to approximate
the integral over a cell in the mesh and then we only need to
evaluate $f(x,y)$ at the mesh points.

Suppose $F_{i,j}$ is computed. The calculation of $F_{i+1,j}$
is then

$$
\begin{align*}
F_{i+1,j} &= F_{i,j} + \int_{x_i}^{x_{i+1}}\int_{y_0}^{y_j} f(x,y)dydx\\
& \approx \Delta x \frac{1}{2}\left(
\int_{y_0}^{y_j} f(x_{i},y)dy
+ \int_{y_0}^{y_j} f(x_{i+1},y)dy\right)
\end{align*}
$$

The integrals in the $y$ direction can be approximated by a Trapezoidal
rule. A similar idea can be used to compute $F_{i,j+1}$. Thereafter,
$F_{i+1,j+1}$ can be computed by adding the integral over the final
corner cell to $F_{i+1,j} + F_{i,j+1} - F_{i,j}$. Carry out the
details of these computations and implement a function that can
return $F_{i,j}$ for all mesh indices $i$ and $j$. Use the
fact that the Trapezoidal rule is exact for linear functions and
write a test function.
Filename: `mesh_calculus_2D`.

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 3: Implement Neumann conditions in 2D
<div id="wave:app:exer:wave2D:Neumann"></div>

Modify the [`wave2D_u0.py`](${src_wave}/wave2D_u0/wave2D_u0.py)
program, which solves the 2D wave equation $u_{tt}=c^2(u_{xx}+u_{yy})$
with constant wave velocity $c$ and $u=0$ on the boundary, to have
Neumann boundary conditions: $\partial u/\partial n=0$.
Include both scalar code (for debugging and reference) and
vectorized code (for speed).

To test the code, use $u=1.2$ as solution ($I(x,y)=1.2$, $V=f=0$, and
$c$ arbitrary), which should be exactly reproduced with any mesh
as long as the stability criterion is satisfied.
Another test is to use the plug-shaped pulse
in the `pulse` function from the section [wave:pde2:software](#wave:pde2:software)
and the [`wave1D_dn_vc.py`](${src_wave}/wave1D/wave1D_dn_vc.py)
program. This pulse
is exactly propagated in 1D if $c\Delta t/\Delta x=1$. Check
that also the 2D program can propagate this pulse exactly
in $x$ direction ($c\Delta t/\Delta x=1$, $\Delta y$ arbitrary)
and $y$ direction ($c\Delta t/\Delta y=1$, $\Delta x$ arbitrary).
Filename: `wave2D_dn`.

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 4: Test the efficiency of compiled loops in 3D
<div id="wave:exer:3D:f77:cy:efficiency"></div>

Extend the `wave2D_u0.py` code and the Cython, Fortran, and C versions to 3D.
Set up an efficiency experiment to determine the relative efficiency of
pure scalar Python code, vectorized code, Cython-compiled loops,
Fortran-compiled loops, and C-compiled loops.
Normalize the CPU time for each mesh by the fastest version.
Filename: `wave3D_u0`.

<!-- --- end exercise --- -->